### Imports

In [ ]:
import cv2
import numpy as np
from src.utilities import debugger,build_montages,print_h,imshow,keep_blobs_by_mask,extract_blobs_on_pattern,get_centroid

from loguru import logger

### def segment_plants(image,debug=True)

In [ ]:
def segment_plants(image,debug=True):
    # Hint: Utilize segmentation class and investigate which segmentation works best for our case and Why?
    # Write Code here...
    plants_segmented = image.copy()


    return plants_segmented

### def assignment(debug= True

In [ ]:
def assignment(debug= True):
    if debug:
        print_h("[Assignment]: Segment max plants from the field\n")
    # Assignment: Cleanly segment maximum no of plants in the image provided.
    #
    #
    # Returns: (BGRA) 4-D image as an ouput. Where first 3 channels are just the original image (BGR)
    #                                                 Last channel is alpha (mask). Indicating segmented plants
    #
    # Hint  : A combination of segmentation algorithm might give a cleaner segmentation result.
    #

    # Reading the image
    field = cv2.imread("Data/drone_view.png")

    if debug:
        imshow("field",field)

    # Task Function
    plants_segmented = segment_plants(field,debug)

    if np.array_equal(plants_segmented,field):
        logger.error("segment_plants() needs to be coded to get the required(Segmented Plant) result.")
        exit(0)

    if debug:
        imshow("plants_segmented",plants_segmented)
        cv2.waitKey(0)

    return plants_segmented

### class segmentation

In [ ]:
class segmentation:
    def __init__(self):
        self.debugger = None

    @staticmethod
    def thresholding(img,type = "binary"):
        if type == "binary":
            img_seg_thresh = cv2.threshold(img,150,255,cv2.THRESH_BINARY)[1]
        elif type == "otsu":
            T,img_seg_thresh = cv2.threshold(img,0,255,cv2.THRESH_OTSU|cv2.THRESH_BINARY)
            print("[INFO] otsu's thresholding value: {}".format(T))
        elif type == "adaptive-mean":
            img_seg_thresh = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
        elif type == 'adaptive-guass':
            img_seg_thresh = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        else:
            print(f"Unknown thresholding type {type}")

        return img_seg_thresh

### def segment_color(self,img,hue_l,hue_h)

In [ ]:
def segment_color(self,img,hue_l,hue_h):
        hls = cv2.cvtColor(img,cv2.COLOR_BGR2HLS)
        hue = hls[:,:,0]
        mask = cv2.inRange(hue,hue_l,hue_h)
        # Perform Closing operation to connect closely disconnected objects
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
        mask = cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
        return mask

### def segment_edges(self,img,thresh_l,thresh_h,aperture)

In [ ]:
def segment_edges(self,img,thresh_l,thresh_h,aperture):
        if len(img.shape)>2:
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        else:
            gray = img
        mask = cv2.Canny(gray,thresh_l,thresh_h,None,aperture)
        return mask

### def segment_kmeans(img,clusters = 2, attempts=10)

In [ ]:
def segment_kmeans(img,clusters = 2, attempts=10):

        # Each channel/feature is placed in a single column
        twoDimage = img.reshape((-1,3))

        twoDimage = np.float32(twoDimage)

        # Defining the criteria for kmean algorithm
        criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_COUNT,10,1.0)

        _,label, center= cv2.kmeans(twoDimage,clusters,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
        # Kmeans- Display Clusters with seperate coloring
        # Output Image : Convert back to Uint8-3dShaped
        center = np.uint8(center)
        res = center[label.flatten()]
        result_image = res.reshape((img.shape))

        return result_image

### def segment(self,img,method="thresholding",type="binary",tune = False)

In [ ]:
def segment(self,img,method="thresholding",type="binary",tune = False):
        if method == "thresholding":
            segmented = self.thresholding(img,type)
        elif method == "color":
            if tune:
                if self.debugger == None: # initialize debugger
                    print("debugger is None ")
                    self.debugger = debugger("Control",["hue_l","hue_h"],[255,255])
                self.debugger.update_variables() # Get updated variables
                hue_l,hue_h= self.debugger.debug_vars[0:2]
            else:
                hue_l= 44;hue_h = 91
            segmented = self.segment_color(img,hue_l,hue_h)
        elif method == "edges":
            if tune:
                if self.debugger == None: #i nitialize debugger
                    self.debugger = debugger("Control (edges)",["thresh_l","thresh_h","aperture"],[255,255,7],[False,False,True])

                self.debugger.update_variables() # Get updated variables
                thresh_l,thresh_h,aperture= self.debugger.debug_vars[0:4]
            else:
                thresh_l=185;thresh_h=255;aperture=5
            segmented = self.segment_edges(img,thresh_l,thresh_h,aperture)
        elif method =="kmeans":
            if tune:
                if self.debugger == None: #i nitialize debugger
                    self.debugger = debugger("Control",["K","attempts"],[10,50])

                self.debugger.update_variables() # Get updated variables
                k,attempts= self.debugger.debug_vars[0:2]
                if k==0:
                    k = 2
                if attempts<10:
                    attempts = 10
            else:
                k = 2; attempts = 10
            segmented = self.segment_kmeans(img,k,attempts)
        else:
            print(f"Unknown Method: {method}")

        return segmented

### def main()

In [ ]:
def main():

    print_h("[main]: Applying different type of Image filters to input and analyzing their effects.")

    img = cv2.imread("Data/boy_who_lived/vignette.jpg",cv2.IMREAD_ANYDEPTH)
    print("img Shape (Input)",img.shape)
    if ( (len(img.shape)==3) and (img.shape[2]==3) ):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    else:
        gray = img

    images = []
    titles = []
    images.append(gray)
    titles.append("img")

    img_segmentor = segmentation()

    # Task 1: Segmentation using Thresholding
    print_h("[Segmentation Using Thresholding]: Investigating the use cases of Simple to adaptive thresholding.")

    # Case: Where thresholding might be the obvious choice
    #   a) Binary
    img_seg_thresh = img_segmentor.segment(gray)
    images.append(img_seg_thresh)
    titles.append("Thresholding (Simple)")



    #   a-2) Otsu
    img_seg_thresh = img_segmentor.segment(gray,type ='otsu')
    images.append(img_seg_thresh)
    titles.append("Thresholding (Otsu)")

    #   b) Adaptive mean
    img_seg_thresh_adaptive = img_segmentor.segment(gray,type="adaptive-mean")
    images.append(img_seg_thresh_adaptive)
    titles.append("Thresholding (adaptive-mean)")


    #   b) Adaptive guassian
    img_seg_thresh_adaptive = img_segmentor.segment(gray,type="adaptive-guass")
    images.append(img_seg_thresh_adaptive)
    titles.append("Thresholding (adaptive-guass)")




    # Displaying image and threshold result
    montage = build_montages(images,None,None,titles,True,True)
    for img in montage:
        cv2.imshow("img",img)
    cv2.waitKey(0)
    cv2.destroyWindow("img")









    # Task 2: Segmentation using Color
    print_h("[Segmentation Using Color]: Performing segmentation based on the selected color.")
    messi_img = cv2.imread("Data\messi5.jpg")

    while(1):
        images.clear()
        titles.clear()
        # Adding image to list of images in montage
        images.append(messi_img)
        titles.append("img (Messi)")

        # 2) Segmentation using color
        img_seg_color = img_segmentor.segment(messi_img,"color",tune=False)
        images.append(img_seg_color)
        titles.append("Segmented (color)")


        # 3) Segmentation using edges
        img_seg_edges = img_segmentor.segment(messi_img,"edges",tune=True)
        images.append(img_seg_edges)
        titles.append("Segmented (edges)")


        # Displaying image and threshold result
        montage = build_montages(images,None,None,titles,True,True)
        for img in montage:
            cv2.imshow("img",img)
        k = cv2.waitKey(1)
        if k==27:
            break
    cv2.destroyAllWindows()
    img_segmentor.debugger = None




    # Task 3: Segmentation using Clustering (kmeans)
    print_h("[Segmentation Using Clustering]: Applying kmeans to divide image into similar regions(clusters).")
    baboon_img = cv2.imread("Data/baboon.jpg")

    while(1):
        images.clear()
        titles.clear()
        images.append(baboon_img)
        titles.append("img (baboon)")

        # Performing kmean segmentation

        img_seg_kmeans = img_segmentor.segment(baboon_img,"kmeans",tune=True)
        images.append(img_seg_kmeans)
        titles.append("Segmented (kmeans)")








        # Displaying image and threshold result
        montage = build_montages(images,None,None,titles,True,True)
        for img in montage:
            cv2.imshow("img",img)

        k = cv2.waitKey(1)
        if k==27:
            break

### Conditional Execution of Functions Based on Readiness Status

In [ ]:
if __name__ == "__main__":
    i_am_ready = False

    if i_am_ready:
        assignment()
    else:
        main()